In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from copy import deepcopy
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

In [41]:
train = pd.read_excel('train_universidad.xlsx',sheet_name=1)
test = pd.read_excel('test_universidad.xlsx')

In [42]:
print(train.columns)
print(test.columns)

Index(['COD_ENCUESTADO', 'Nombre Campus', 'NIVEL ACTUAL', 'Clave de carrera',
       'Ciclo', 'COMENTARIO', 'IND_GEA', 'IND_DELEGADO',
       'CANT_CURSOS_MATRICU_SIN_INGLES', 'UOD_depostista_ind_deportista',
       'NPS'],
      dtype='object')
Index(['COD_ENCUESTADO', 'Nombre Campus', 'NIVEL ACTUAL', 'Clave de carrera',
       'Ciclo', 'COMENTARIO', 'IND_GEA', 'IND_DELEGADO',
       'CANT_CURSOS_MATRICU_SIN_INGLES', 'UOD_depostista_ind_deportista'],
      dtype='object')


In [43]:
train.head()

,COD_ENCUESTADO,Nombre Campus,NIVEL ACTUAL,Clave de carrera,Ciclo,COMENTARIO,IND_GEA,IND_DELEGADO,CANT_CURSOS_MATRICU_SIN_INGLES,UOD_depostista_ind_deportista,NPS
0,13501,1,AC,3,2,Me gusta la u es paja bro y soy pito,NaN,Delegado,6.0,NaN,3
1,23622,3,AC,25,1,El metodo de blended no le hace bien a todos,NaN,Delegado,5.0,NaN,3
2,8354,4,AC,31,1,"Los profesores, sus métodos de enseñanza bes ...",NaN,NaN,5.0,NaN,4
3,17745,4,AC,28,6,Porque posee gran mayoría de profesores espec...,GEA,NaN,6.0,NaN,3
4,10867,3,AC,34,1,La pencion,NaN,NaN,6.0,NaN,3


In [44]:
test['NPS']=''
train['train']=1
test['train']=0

In [45]:
dataset=pd.concat([train,test])#Junto todo en un dataset

In [46]:
dataset.head()

,COD_ENCUESTADO,Nombre Campus,NIVEL ACTUAL,Clave de carrera,Ciclo,COMENTARIO,IND_GEA,IND_DELEGADO,CANT_CURSOS_MATRICU_SIN_INGLES,UOD_depostista_ind_deportista,NPS,train
0,13501,1,AC,3,2,Me gusta la u es paja bro y soy pito,NaN,Delegado,6.0,NaN,3,1
1,23622,3,AC,25,1,El metodo de blended no le hace bien a todos,NaN,Delegado,5.0,NaN,3,1
2,8354,4,AC,31,1,"Los profesores, sus métodos de enseñanza bes ...",NaN,NaN,5.0,NaN,4,1
3,17745,4,AC,28,6,Porque posee gran mayoría de profesores espec...,GEA,NaN,6.0,NaN,3,1
4,10867,3,AC,34,1,La pencion,NaN,NaN,6.0,NaN,3,1


In [47]:
dataset.shape

(28427, 12)

In [48]:
from nltk.corpus import stopwords
sw = stopwords.words("spanish")


In [56]:
#sw.append() #agrega la lista en su forma natural
#sw.extend('')#agrega una lista con valores separados
#sw.remove('no')


In [50]:
sw

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosostros',
 'vosostras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro',
 'v

### 1)Vuelvo a miniscula

In [64]:
dataset['lower']=dataset['COMENTARIO'].str.lower()

In [65]:
dataset['lower'].head()

0                 me gusta la u es paja bro y soy pito
1         el metodo de blended no le hace bien a todos
2     los profesores, sus métodos de enseñanza bes ...
3     porque posee gran mayoría de profesores espec...
4                                           la pencion
Name: lower, dtype: object

In [66]:
!pip3 install pattern

In [67]:
from nltk.stem.porter import PorterStemmer
import pattern.es as lemEsp
porter = PorterStemmer()

def tokenizer(text):
    return text.split()
def tokenizer_porter(text):
    return ' '.join([porter.stem(word) for word in text.split()])
def tokenizer_Lem(text):
    return ' '.join(lemEsp.Sentence(lemEsp.parse(text, lemmata=True)).lemmata)


### 2)Tokenizo

In [68]:
dataset['tokenize'] = dataset['lower'].apply(tokenizer)
dataset['tokenize'].head()

0      [me, gusta, la, u, es, paja, bro, y, soy, pito]
1    [el, metodo, de, blended, no, le, hace, bien, ...
2    [los, profesores,, sus, métodos, de, enseñanza...
3    [porque, posee, gran, mayoría, de, profesores,...
4                                        [la, pencion]
Name: tokenize, dtype: object

### 3) Le quito los stopwords

In [69]:
dataset['stopwords']=dataset['tokenize'].apply(lambda x: [word for word in x if word not in (sw)])
dataset['stopwords'].head()

0                          [gusta, u, paja, bro, pito]
1                    [metodo, blended, no, hace, bien]
2    [profesores,, métodos, enseñanza, bes, buena.,...
3    [posee, gran, mayoría, profesores, especializa...
4                                            [pencion]
Name: stopwords, dtype: object

In [58]:
dataset['tokenize']

0         [me, gusta, la, u, es, paja, bro, y, soy, pito]
1       [el, metodo, de, blended, no, le, hace, bien, ...
2       [los, profesores,, sus, métodos, de, enseñanza...
3       [porque, posee, gran, mayoría, de, profesores,...
4                                           [la, pencion]
5       [mayor, espacio, en, los, estacionamientos, ya...
6                                    [calidad, educativa]
7                  [la, enseñanza, y, buena, metodología]
8       [tiene, tecnología, de, qltq, calidad, y, tien...
9       [no, es, posible, que, para, tener, que, matri...
10      [por, el, excelente, servicio, académico., tam...
11      [quizas, en, exigencia, y, en, la, publicacion...
12      [cuando, se, bloque, la, contraseña, no, manda...
13                                          [mejorar, it]
14            [mantener, los, costos, de, las, pensiones]
15         [en, el, mantenimiento, de, baños, y, salones]
16      [primero,, cambiar, el, horario, de, estudios,...
17      [aún, 

In [70]:
dataset['stopwords']=dataset['stopwords'].apply(lambda x: ' '.join(x))
dataset['stopwords'].head()

0                                gusta u paja bro pito
1                          metodo blended no hace bien
2    profesores, métodos enseñanza bes buena. uso t...
3    posee gran mayoría profesores especializados e...
4                                              pencion
Name: stopwords, dtype: object

### 4)Vuelvo a su palabra "raiz"

In [71]:
dataset['stopwords_porter'] = dataset['stopwords'].apply(tokenizer_porter)
dataset['stopwords_porter'].head()

0                                gusta u paja bro pito
1                            metodo blend no hace bien
2    profesores, método enseñanza be buena. uso tec...
3     pose gran mayoría profesor especializado enseñan
4                                              pencion
Name: stopwords_porter, dtype: object

### 4)Vuelvo a su palabra "lematizada"

In [72]:
########cambio
dataset['stopwords_Lem'] = dataset['stopwords'].apply(tokenizer_Lem)
dataset['stopwords_Lem'].head()

0                               gustar u paja bro pito
1                         metodo blended no hacer bien
2    profesor , método enseñanza bes bueno .\nuso t...
3    poseer gran mayoría profesor especializados en...
4                                              pencion
Name: stopwords_Lem, dtype: object

### 5) Le doy importancia a las palabras

In [74]:
vect = TfidfVectorizer(ngram_range=(1, 3),max_features=1500)
prueba= vect.fit_transform(dataset['stopwords_Lem'])

In [75]:
pd.DataFrame(vect.fit_transform(dataset['stopwords_Lem']).toarray(), columns=vect.get_feature_names())

,10,100,11,15,20,30,abastecer,abrir,abrir curso,abrir salón,...,wifi lento,wifi no,ya,zona,ámbito,área,área estudio,área verde,último,único
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.199156,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.301952,0.192151,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


### 6)Separao mis datos de train y de prueba

In [76]:
#logistic
mstrain = prueba[:20000,]
mstrain_y = dataset.iloc[:20000,-7]
mstest = prueba[20000:,]

In [77]:
dataset.iloc[:20000,-7]

0        3
1        3
2        4
3        3
4        3
5        3
6        3
7        4
8        4
9        2
10       4
11       2
12       2
13       3
14       3
15       3
16       1
17       3
18       3
19       1
20       4
21       3
22       3
23       3
24       3
25       4
26       4
27       3
28       4
29       3
        ..
19970    4
19971    4
19972    3
19973    2
19974    3
19975    3
19976    4
19977    4
19978    4
19979    3
19980    4
19981    4
19982    3
19983    2
19984    3
19985    3
19986    3
19987    3
19988    4
19989    3
19990    3
19991    2
19992    4
19993    4
19994    4
19995    4
19996    3
19997    3
19998    2
19999    4
Name: NPS, Length: 20000, dtype: object

In [78]:
#Naive
mstrain1 = prueba[:20000,]
mstrain_y1 = dataset.iloc[:20000,-7]
mstest1 = prueba[20000:,]


In [79]:
dataset.iloc[:20000,-7]

0        3
1        3
2        4
3        3
4        3
5        3
6        3
7        4
8        4
9        2
10       4
11       2
12       2
13       3
14       3
15       3
16       1
17       3
18       3
19       1
20       4
21       3
22       3
23       3
24       3
25       4
26       4
27       3
28       4
29       3
        ..
19970    4
19971    4
19972    3
19973    2
19974    3
19975    3
19976    4
19977    4
19978    4
19979    3
19980    4
19981    4
19982    3
19983    2
19984    3
19985    3
19986    3
19987    3
19988    4
19989    3
19990    3
19991    2
19992    4
19993    4
19994    4
19995    4
19996    3
19997    3
19998    2
19999    4
Name: NPS, Length: 20000, dtype: object

In [80]:
mstest

<8427x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 53353 stored elements in Compressed Sparse Row format>

In [81]:
mstrain_y = mstrain_y.astype('int')#vuelvo hacer entere la columna train

In [82]:
mstrain_y1 = mstrain_y1.astype('int')#vuelvo hacer entere la columna train

## Using Sklearn

In [83]:
from sklearn.neighbors import KNeighborsClassifier

In [84]:
knn = KNeighborsClassifier(n_neighbors=10)

In [85]:
codigo = dataset.iloc[20000:,0]

In [86]:
knn.fit(mstrain, mstrain_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [135]:
y_pred_prob1 = knn.predict_proba(mstest) #mstest = dataset['tokenizer_porter'][20000:]

In [136]:
Test1=pd.DataFrame(y_pred_prob1)

In [137]:
Test1

,0,1,2,3
0,0.2,0.1,0.4,0.3
1,0.1,0.3,0.4,0.2
2,0.0,0.0,0.2,0.8
3,0.0,0.0,0.0,1.0
4,0.1,0.2,0.2,0.5
5,0.0,0.0,0.2,0.8
6,0.0,0.5,0.5,0.0
7,0.0,0.2,0.3,0.5
8,0.0,0.0,0.3,0.7
9,0.0,0.1,0.4,0.5


In [138]:
Test1['COD_ENCUESTADO']=codigo

In [140]:
Test1.head(10)

,0,1,2,3,COD_ENCUESTADO
0,0.2,0.1,0.4,0.3,15118
1,0.1,0.3,0.4,0.2,6715
2,0.0,0.0,0.2,0.8,27492
3,0.0,0.0,0.0,1.0,29693
4,0.1,0.2,0.2,0.5,29677
5,0.0,0.0,0.2,0.8,26412
6,0.0,0.5,0.5,0.0,8181
7,0.0,0.2,0.3,0.5,22242
8,0.0,0.0,0.3,0.7,9485
9,0.0,0.1,0.4,0.5,7671


In [141]:
Test1=Test1.rename({'0':'NPS1','1':'NPS2','2':'NPS3','3':'NPS4'})

In [146]:
Test1

,COD_ENCUESTADO,NPS1,NPS2,NPS3,NPS4
0,15118,0.2,0.1,0.4,0.3
1,6715,0.1,0.3,0.4,0.2
2,27492,0.0,0.0,0.2,0.8
3,29693,0.0,0.0,0.0,1.0
4,29677,0.1,0.2,0.2,0.5
5,26412,0.0,0.0,0.2,0.8
6,8181,0.0,0.5,0.5,0.0
7,22242,0.0,0.2,0.3,0.5
8,9485,0.0,0.0,0.3,0.7
9,7671,0.0,0.1,0.4,0.5


In [143]:
Test1.columns= ['NPS1','NPS2','NPS3','NPS4','COD_ENCUESTADO']

In [144]:
VAR = ['COD_ENCUESTADO','NPS1','NPS2','NPS3','NPS4']

In [145]:
Test1 = Test1[VAR]

In [98]:
Test1.to_csv('primer_modelo.csv', index=False)

## Naive Bayes

In [99]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score

/Users/bryanttello/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [147]:
# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(mstrain1, mstrain_y1)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [148]:
Test1=pd.DataFrame(nb.predict_proba(mstest1))

In [149]:
nb1 = MultinomialNB()
nb1.fit(mstrain, mstrain_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [150]:
Test1=pd.DataFrame(nb1.predict_proba(mstest))

In [152]:
Test1['COD_ENCUESTADO']=codigo

In [153]:
Test1.head(10)

,0,1,2,3,COD_ENCUESTADO
0,0.046609,0.247571,0.683408,0.022412,15118
1,0.098795,0.319938,0.500499,0.080769,6715
2,0.000644,0.001534,0.008589,0.989233,27492
3,0.002494,0.008711,0.021558,0.967237,29693
4,0.135656,0.336748,0.470597,0.056999,29677
5,0.006327,0.035138,0.155454,0.803081,26412
6,0.149102,0.489922,0.327060,0.033917,8181
7,0.099884,0.359958,0.493425,0.046733,22242
8,0.010849,0.037588,0.158004,0.793559,9485
9,0.039111,0.072834,0.306311,0.581745,7671


In [154]:
Test1=Test1.rename({'0':'NPS1','1':'NPS2','2':'NPS3','3':'NPS4'})

In [155]:
Test1.head(10)

,0,1,2,3,COD_ENCUESTADO
0,0.046609,0.247571,0.683408,0.022412,15118
1,0.098795,0.319938,0.500499,0.080769,6715
2,0.000644,0.001534,0.008589,0.989233,27492
3,0.002494,0.008711,0.021558,0.967237,29693
4,0.135656,0.336748,0.470597,0.056999,29677
5,0.006327,0.035138,0.155454,0.803081,26412
6,0.149102,0.489922,0.327060,0.033917,8181
7,0.099884,0.359958,0.493425,0.046733,22242
8,0.010849,0.037588,0.158004,0.793559,9485
9,0.039111,0.072834,0.306311,0.581745,7671


In [156]:
Test1.columns= ['NPS1','NPS2','NPS3','NPS4','COD_ENCUESTADO']

In [157]:
VAR = ['COD_ENCUESTADO','NPS1','NPS2','NPS3','NPS4']

In [158]:
Test1 = Test1[VAR]

In [159]:
Test1.to_csv('segundo.csv', index=False)